# Desafío 3

### Consigna
- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.

_Se realuizarán las consignas tanto para un modelo de lenguaje de caracteres como uno de palabras, como hemos visto en clase._

#### Código Preliminar
_<span style="font-size:smaller;">Imports y configuración.</span>_

In [87]:
#####  Código Preliminar  #####
import os
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr



from tensorflow.keras.utils import pad_sequences
import keras
from keras.layers import SimpleRNN, Dense, Embedding
from keras.models import Model, Sequential
from keras.models import load_model

plt.style.use('dark_background')
books_directory = "data/d3/books"

#### Selección del Corpus
_<span style="font-size:smaller;">Volveremos a usar libros del Gudenberg Project, esta vez una mayor cantidad.</span>_

In [ ]:
book_ids = pd.read_csv("data/d3/books_ids.csv")
total_books = len(book_ids)

for book_id in tqdm(book_ids['book_id'], desc="Descargando libros. . .", total=total_books):
    url = f"https://www.gutenberg.org/ebooks/{book_id}.txt.utf-8"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Lanza un error si la descarga falla
        with open(os.path.join(books_directory, f"{book_id}.txt"), 'w', encoding='utf-8') as file:
            file.write(response.text)
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar el libro {book_id}: {e}")

## Modelo de Caracteres

### Pre-procesado del Corpus
_<span style="font-size:smaller;">Unificamos los documentos en un solo corpus y lo tokenizamos</span>_

In [4]:
# Leemos todos los textos y los almacenamos en una lista
corpus = []
for filename in os.listdir(books_directory):
    if filename.endswith(".txt"):
        file_path = os.path.join(books_directory, filename)
        with open(file_path, 'r', encoding='utf-8-sig') as file:
            # Leer el contenido del archivo y agregarlo a la lista
            content = file.read()
            corpus.append(content)

# Concatenamos todos los textos en una sola cadena
corpus_text = ' '.join(corpus)

max_context_size = 100
chars_vocab = set(corpus_text)
print(f"Tenemos un corpus de {len(corpus_text)} caracteres")
print(f"Y un vocabulario de {len(chars_vocab)}.")


Tenemos un corpus de 4024032 caracteres
Y un vocabulario de 118.


In [8]:
char_to_idx = {char: idx for idx, char in enumerate(sorted(chars_vocab))}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}


In [ ]:
import numpy as np
from tqdm import tqdm

max_context_size = 100  # El tamaño de la secuencia de entrada
corpus_length = len(corpus_text)
vocab_size = len(chars_vocab)

# Inicializar matrices de entrada y salida
input_sequences = np.zeros((corpus_length - max_context_size, max_context_size), dtype=np.int32)
target_sequences = np.zeros(corpus_length - max_context_size, dtype=np.int32)

# Rellenar matrices con índices correspondientes
for i in tqdm(range(corpus_length - max_context_size)):
    input_sequences[i] = [char_to_idx[char] for char in corpus_text[i: i + max_context_size]]
    target_sequences[i] = char_to_idx[corpus_text[i + max_context_size]]

print(f"Generamos {input_sequences.shape[0]} secuencias de entrada.")


### Estructuración del Dataset
_<span style="font-size:smaller;">Dividimos en train y test como vimos en clase.</span>_

In [ ]:
X = np.array(input_sequences)
y = np.array(target_sequences)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding

vocab_size = len(chars_vocab)
embedding_dim = 50  # Puedes ajustar este valor

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_context_size))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


### Entrenamiento

_<span style="font-size:smaller;">El resultado se guardará en la carpeta /models.</span>_

In [ ]:
model.fit(X, y, epochs=4, batch_size=64)
model.save('models/lang_char.keras')

### Carga del modelo

_<span style="font-size:smaller;">Para no entrenar en cada ejecución.</span>_

In [2]:
model = load_model('models/lang_char.keras')

### Generación de Secuencias - Beam Search

In [55]:
# funcionalidades para hacer encoding y decoding

def encode(input_text):
    encoded = [char_to_idx[char] for char in input_text]
    return np.array([encoded])  # Devolver una matriz 2D con forma (1, sequence_length)

def decode(seq):
    return ''.join([idx_to_char[ch] for ch in seq])

In [68]:
def beam_search(model, num_beams, num_words, input, temp=1.0, mode='det'):
    # Primera iteración

    # Codificar
    encoded = encode(input)

    # Primera predicción
    y_hat = model.predict(encoded, verbose=0)[0, :]

    # Obtener el tamaño del vocabulario
    vocab_size = y_hat.shape[0]

    # Inicializar historial
    history_probs = [0] * num_beams
    history_tokens = [encoded[0]] * num_beams

    # Seleccionar candidatos
    history_probs, history_tokens = select_candidates([y_hat],
                                                      num_beams,
                                                      vocab_size,
                                                      history_probs,
                                                      history_tokens,
                                                      temp,
                                                      mode)

    # Bucle de búsqueda en haz
    for i in range(num_words - 1):
        preds = []

        for hist in history_tokens:
            # Actualizar secuencia de tokens
            input_update = np.array([hist[-max_context_size:]])

            # Predicción
            y_hat = model.predict(input_update, verbose=0)[0, :]

            # Aplicar la temperatura a las probabilidades antes de añadir a preds
            y_hat = np.log(y_hat + 1e-10) / temp
            y_hat = np.exp(y_hat) / np.sum(np.exp(y_hat))  # Softmax con temperatura

            preds.append(y_hat)

        history_probs, history_tokens = select_candidates(preds,
                                                          num_beams,
                                                          vocab_size,
                                                          history_probs,
                                                          history_tokens,
                                                          temp,
                                                          mode)

    return history_tokens

In [81]:
# Observemos salidas con distintas temperaturas
print(decode(beam_search(model,num_beams=10,num_words=15,input="He is ", temp=1)[0]))
print(decode(beam_search(model,num_beams=10,num_words=15,input="He is ", temp=0.5)[0]))
print(decode(beam_search(model,num_beams=10,num_words=15,input="He is ", temp=2)[0]))
print(decode(beam_search(model,num_beams=10,num_words=15,input="He is ", temp=10)[0]))
print(decode(beam_search(model,num_beams=10,num_words=15,input="He is ", temp=20)[0]))




He is another, while 
He is they were they 
He is thought though 
He is another,” said 
He is anotherwisfulle


In [60]:
def greedy_search(model, input_text, num_words, temp=1.0):
    # Codificar el texto de entrada
    encoded = encode(input_text)
    
    # Inicializar la secuencia generada
    generated_sequence = list(encoded[0])

    for _ in range(num_words):
        # Preparar la entrada para el modelo
        input_sequence = np.array([generated_sequence[-max_context_size:]])
        
        # Obtener la predicción del modelo
        y_hat = model.predict(input_sequence, verbose=0)[0, :]
        
        # Aplicar la temperatura a las probabilidades
        y_hat = np.log(y_hat + 1e-10) / temp
        y_hat = np.exp(y_hat) / np.sum(np.exp(y_hat))  # Softmax con temperatura

        # Seleccionar el índice con la mayor probabilidad
        next_char_idx = np.argmax(y_hat)
        
        # Agregar el siguiente carácter a la secuencia generada
        generated_sequence.append(next_char_idx)

    return generated_sequence


In [85]:
# Definir el texto de entrada y el número de caracteres a generar
input_text = "He is "
num_words = 20

# Ejecutamos la greedy
result_sequence = greedy_search(model, input_text, num_words, 0.1)

# Decodificamos la secuencia generada
generated_text = decode(result_sequence)
print(generated_text)


He is a stranger of the

s


## Modelo de Palabras

### Segmentación / Tokenización

In [90]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Creamos un tokenizer para convertir las palabras a índices
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus_text])

# Convertir el texto completo en una secuencia de índices de palabras
sequences = tokenizer.texts_to_sequences([corpus_text])[0]

# Obtener el tamaño del vocabulario
vocab_size = len(tokenizer.word_index) + 1  # +1 por el token de padding

print(f"El tamaño del vocabulario es: {vocab_size}")

El tamaño del vocabulario es: 31515


### Generación de Secuencias

In [91]:
max_context_size = 10  # Tamaño de la secuencia de entrada

input_sequences = []
target_sequences = []

for i in range(0, len(sequences) - max_context_size):
    input_seq = sequences[i: i + max_context_size]
    target_word = sequences[i + max_context_size]
    input_sequences.append(input_seq)
    target_sequences.append(target_word)

# Convertir a arrays de NumPy
X = np.array(input_sequences)
y = np.array(target_sequences)

print(f"Generamos {len(input_sequences)} secuencias de entrada.")


Generamos 709430 secuencias de entrada.


### Construcción del modelo

In [93]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

embedding_dim = 50  # Puedes ajustar este valor

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


### Entrenamiento

In [94]:
model.fit(X, y, epochs=20, batch_size=64)
model.save('models/lang_word.keras')

Epoch 1/20
11085/11085 ━━━━━━━━━━━━━━━━━━━━ 526s 47ms/step - accuracy: 0.0634 - loss: 6.9927
Epoch 2/20
11085/11085 ━━━━━━━━━━━━━━━━━━━━ 519s 47ms/step - accuracy: 0.1140 - loss: 6.0684
Epoch 3/20
11085/11085 ━━━━━━━━━━━━━━━━━━━━ 527s 47ms/step - accuracy: 0.1284 - loss: 5.7463
Epoch 4/20
11085/11085 ━━━━━━━━━━━━━━━━━━━━ 520s 47ms/step - accuracy: 0.1393 - loss: 5.5229
Epoch 5/20
11085/11085 ━━━━━━━━━━━━━━━━━━━━ 521s 47ms/step - accuracy: 0.1475 - loss: 5.3444
Epoch 6/20
11085/11085 ━━━━━━━━━━━━━━━━━━━━ 522s 47ms/step - accuracy: 0.1551 - loss: 5.2018
Epoch 7/20
11085/11085 ━━━━━━━━━━━━━━━━━━━━ 523s 47ms/step - accuracy: 0.1608 - loss: 5.0886
Epoch 8/20
11085/11085 ━━━━━━━━━━━━━━━━━━━━ 523s 47ms/step - accuracy: 0.1674 - loss: 4.9732
Epoch 9/20
11085/11085 ━━━━━━━━━━━━━━━━━━━━ 523s 47ms/step - accuracy: 0.1734 - loss: 4.8620
Epoch 10/20
11085/11085 ━━━━━━━━━━━━━━━━━━━━ 523s 47ms/step - accuracy: 0.1795 - loss: 4.7477
Epoch 11/20
11085/11085 ━━━━━━━━━━━━━━━━━━━━ 524s 47ms/step - accurac

### Carga del modelo

In [ ]:
model = load_model('models/lang_word.keras')

### Generación de Secuencias

In [98]:
def beam_search(model, num_beams, num_words, input_text, temp=1.0, mode='det'):
    # Codificar el texto de entrada
    encoded = tokenizer.texts_to_sequences([input_text])[0]
    generated_sequence = list(encoded)

    # Inicializar historial
    history_probs = [0] * num_beams
    history_tokens = [generated_sequence] * num_beams

    for _ in range(num_words):
        preds = []

        for hist in history_tokens:
            input_sequence = pad_sequences([hist[-max_context_size:]], maxlen=max_context_size)
            
            # Obtener la predicción del modelo
            y_hat = model.predict(input_sequence, verbose=0)[0, :]
            
            # Aplicar la temperatura a las probabilidades
            y_hat = np.log(y_hat + 1e-10) / temp
            y_hat = np.exp(y_hat) / np.sum(np.exp(y_hat))  # Softmax con temperatura

            preds.append(y_hat)

        history_probs, history_tokens = select_candidates(preds,
                                                          num_beams,
                                                          vocab_size,
                                                          history_probs,
                                                          history_tokens,
                                                          temp,
                                                          mode)

    return history_tokens

# Función de decodificación para convertir de índices a texto
def decode(sequence):
    return ' '.join([tokenizer.index_word[idx] for idx in sequence if idx in tokenizer.index_word])


In [99]:
num_beams = 3

# Generación con temperatura 1.0 (predeterminada)
result_tokens = beam_search(model, num_beams, num_words, input_text, temp=1.0)
generated_text_default = decode(result_tokens[0])
print("Beam Search (Temp = 1.0):")
print(generated_text_default)

# Generación con temperatura 0.5
result_tokens = beam_search(model, num_beams, num_words, input_text, temp=0.5)
generated_text_low_temp = decode(result_tokens[0])
print("\nBeam Search (Temp = 0.5):")
print(generated_text_low_temp)

# Generación con temperatura 1.5
result_tokens = beam_search(model, num_beams, num_words, input_text, temp=1.5)
generated_text_high_temp = decode(result_tokens[0])
print("\nBeam Search (Temp = 1.5):")
print(generated_text_high_temp)


Beam Search (Temp = 1.0):
he is a weary ” —sir henry eyeing him as a bridal and the upper end of the country and the sailors

Beam Search (Temp = 0.5):
he is a weary ” —sir henry eyeing him as a bridal and the upper end of the country and the sailors

Beam Search (Temp = 1.5):
he is a weary ” —sir henry eyeing him as a bridal and the upper end of the country and the sailors
